# Multiscale MILP

This is a continuation of 'The Scheduling Example', refer to learn the basics on how Energia models processes. 

In this example, we add another Process [Solar PV] and Storage [Li-ion Battery]. Technology choice is modeled using binaries. Moreover, the model is multiscale as the operational capacities are decision variables. 

In [25]:
from energia import *

m = Model('example2')
m.q = Periods()
m.y = 4 * m.q
m.usd = Currency()

## Resources

### Conveniently declaring components

Use m.declare(\<Object Type\>, \<list of names\>) to declare a large number of objects in one step.

In [26]:
m.declare(Resource, ['power', 'wind', 'solar'])

### Set bounds on Resource flows

Unlike wind which has bound on the total consumption, we set a daily limit on solar energy. The same bound is repeated in each quarter. The following constraints are written.

$\mathbf{cons}_{solar, network, quarter_0} \leq 100$

$\mathbf{cons}_{solar, network, quarter_1} \leq 100$

$\mathbf{cons}_{solar, network, quarter_2} \leq 100$

$\mathbf{cons}_{solar, network, quarter_3} \leq 100$

In [27]:
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

⚖   Initiated solar balance in (l0, q)                                      ⏱ 0.0001 s
🔗  Bound [≤] solar consume in (l0, q)                                       ⏱ 0.0012 s
⚖   Initiated wind balance in (l0, y)                                       ⏱ 0.0001 s
🔗  Bound [≤] wind consume in (l0, y)                                        ⏱ 0.0009 s
⚖   Initiated power balance in (l0, q)                                      ⏱ 0.0001 s
🔗  Bound [≥] power release in (l0, q)                                       ⏱ 0.0012 s


## Operations 

### Capacity as a variable 

Here we want the optimization problem to determine the optimal capacity. Moreover, we set binaries to avoid the lower bound being adhered to if the process is not set up. 

If the bounds are meant to be compulsory limits, skip the .x 

In [28]:
m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.capacity.show()

🔗  Bound [≤] wf capacity in (l0, y)                                         ⏱ 0.0003 s
🔗  Bound [≥] wf capacity in (l0, y)                                         ⏱ 0.0003 s


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Unlike in Example 1, where the capacity was know, capacity is a variable here. 

Moreover, the expenditure associated with operating and capacitating are different

In [29]:
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49
m.operate.show(True)

🔗  Bound [≤] wf operate in (l0, q)                                          ⏱ 0.0003 s
🧭  Mapped time for operate (wf, l0, q) ⟺ (wf, l0, y)                        ⏱ 0.0002 s


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [30]:
m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10

m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

🔗  Bound [≤] pv capacity in (l0, y)                                         ⏱ 0.0005 s
🔗  Bound [≥] pv capacity in (l0, y)                                         ⏱ 0.0002 s
🔗  Bound [≤] pv operate in (l0, q)                                          ⏱ 0.0003 s
🧭  Mapped time for operate (pv, l0, q) ⟺ (pv, l0, y)                        ⏱ 0.0002 s


### Storage Operation

energia now allows storing to require the use of other resources, example power for hydrogen cryogenic storage. 

Provide an equation similar to Process, in this case the basis is the stored resource 
If no other resource is provided, it is assumed to be the charging/discharging efficiency

Note that the following are created internally: 
1. auxilary resource  with name resource.stored 
2. charging and discharging processes as storage.charge and storage.discharge 

The parameters for each of these can be set individually, thus allowing for a wide range of modeling approaches 

In [31]:
m.lii = Storage()

m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10

# m.lii.capacity >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432

m.lii.inventory[m.usd.spend] == 2000


🔗  Bound [≤] lii.stored invcapacity in (l0, y)                              ⏱ 0.0003 s
🔗  Bound [≥] lii.stored invcapacity in (l0, y)                              ⏱ 0.0002 s


## Locating Operations

Operations can be located as 

operation.locate(\<list of locations\>)

or 

m.location.operations(\<list of operations\>)

They both do the same thing 

In [32]:
m.pv.locate(m.network)
m.network.locate(m.wf, m.lii)

💡  Assumed pv capacity unbounded in (l0, y)                                 ⏱ 0.0001 s
💡  Assumed pv operate bounded by capacity in (l0, q)                        ⏱ 0.0001 s
⚖   Updated power balance with produce(power, l0, q, operate, pv)           ⏱ 0.0001 s
⚖   Updated solar balance with expend(solar, l0, q, operate, pv)            ⏱ 0.0001 s
🏭  Operating streams introduced for pv in l0                                ⏱ 0.0020 s
🏗   Construction streams introduced for pv in l0                            ⏱ 0.0000 s
🌍  Located pv in l0                                                         ⏱ 0.0035 s
💡  Assumed wf capacity unbounded in (l0, y)                                 ⏱ 0.0001 s
💡  Assumed wf operate bounded by capacity in (l0, q)                        ⏱ 0.0001 s
⚖   Updated power balance with produce(power, l0, q, operate, wf)           ⏱ 0.0001 s
⚖   Updated wind balance with expend(wind, l0, y, operate, wf)              ⏱ 0.0001 s
🏭  Operating streams introduced for wf in l

## Inventory Balance

Inventory is passed on from one time period (t - 1) to the next (t) and hence features in the general resource balance for resource.stored 

In [33]:
m.inventory.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Optimize!

In [34]:
m.usd.spend.opt()

🧭  Mapped samples for spend (usd, l0, y, capacity, wf) ⟺ (usd, l0, y)       ⏱ 0.0002 s
🧭  Mapped samples for spend (usd, l0, y, operate, wf) ⟺ (usd, l0, y)        ⏱ 0.0002 s
🧭  Mapped samples for spend (usd, l0, y, capacity, pv) ⟺ (usd, l0, y)       ⏱ 0.0003 s
🧭  Mapped samples for spend (usd, l0, y, operate, pv) ⟺ (usd, l0, y)        ⏱ 0.0002 s
🧭  Mapped samples for spend (usd, l0, y, invcapacity, lii.stored) ⟺ (usd, l0, y) ⏱ 0.0002 s
🧭  Mapped samples for spend (usd, l0, y, inventory, lii.stored) ⟺ (usd, l0, y) ⏱ 0.0002 s
📝  Generated Program(example2).mps                                          ⏱ 0.0053 s


Read MPS format model from file Program(example2).mps
Reading time = 0.00 seconds
PROGRAM(EXAMPLE2): 83 rows, 78 columns, 196 nonzeros


📝  Generated gurobipy model. See .formulation                               ⏱ 0.0073 s


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 83 rows, 78 columns and 196 nonzeros
Model fingerprint: 0xc4ff6a31
Variable types: 75 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 4e+02]
Presolve removed 71 rows and 66 columns
Presolve time: 0.00s
Presolved: 12 rows, 12 columns, 34 nonzeros
Variable types: 12 continuous, 0 integer (0 binary)

Root relaxation: objective 3.006497e+08, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3.006497e+08 3.0065e+08  0.0

📝  Generated Solution object for Program(example2). See .solution           ⏱ 0.0004 s
✅  Program(example2) optimized using gurobi. Display using .output()        ⏱ 0.0166 s


In [35]:
m.show(True)

# Mathematical Program for Program(example2)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Solution

### Inventory Profiles

The inventory maintained in each time period is:

In [36]:
m.lii.discharge.production.balance

{power: 1.0, lii.stored: -1.1111111111111112}

In [37]:
m.proces = Process()

In [38]:
m.proces(m.power) == 20

η(proces, power)

In [39]:
m.proces.production.hold

In [40]:
m.inventory.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount charged into inventory is:

In [41]:
m.produce(m.lii.stored, m.lii.charge.operate, m.q).output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [42]:
m.power.lii

η(lii.discharge, power)

The amount discharged from inventory is:

In [43]:
m.produce(m.power, m.lii.discharge.operate, m.q).output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Integer Decisions 

All the operations are setup in this case

In [44]:
m.capacity.reporting.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [45]:
m.capacity.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>